## The pipeline of only use the blurry Stomata to train the Yolov8

Only use classes Stomata.blurry and Stomata complete and blurry complete .

Degin a dataaugment to let the model work better.

In [ ]:
from ultralytics import YOLO
import torch
import cv2



In [ ]:

class_name_to_id_mapping = {
    "complete": 0,
    "blurry.complete": 2, 
            } # leaves hair

class_id_to_name_mapping = dict(zip(class_name_to_id_mapping.values(), class_name_to_id_mapping.keys()))






model = YOLO("yolov8m.pt")


